## Fake News Classifier Using LSTM

In [5]:
import pandas as pd

In [9]:
data=pd.read_csv('C:\\Users\\KUMAR HIREMATH\\Downloads\\train.csv')

In [10]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
data=data.dropna()

In [12]:
x=data.drop('label',axis=1)
y=data.label

In [13]:
x.shape

(18285, 4)

In [14]:
y.shape

(18285,)

In [15]:
import tensorflow as tf

In [16]:
tf.__version__

'2.4.1'

In [17]:
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [18]:
message=x.copy()

In [19]:
message.reset_index(inplace=True)

In [20]:
import nltk 
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\KUMAR
[nltk_data]     HIREMATH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(message)):
    review=re.sub('[^a-zA-Z]',' ',message['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [23]:
corpus[6]

'beno hamon win french socialist parti presidenti nomin new york time'

In [24]:
voc_size=5000

In [26]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[829, 4353, 4483, 649, 389, 423, 4437, 4181, 1745, 1385],
 [1235, 2058, 521, 3784, 4293, 4275, 2815],
 [978, 517, 3512, 4522],
 [1967, 779, 1484, 3847, 366, 246],
 [4606, 4293, 3574, 1006, 2223, 4167, 4293, 948, 2094, 3187],
 [152,
  208,
  1982,
  2007,
  1950,
  1282,
  1481,
  2720,
  3518,
  376,
  1350,
  2420,
  2985,
  3745,
  2815],
 [2888, 4338, 4075, 2343, 3643, 25, 4880, 3031, 389, 433, 1627],
 [1480, 3274, 1250, 4617, 3585, 2967, 1282, 4504, 389, 433, 1627],
 [4425, 2159, 1681, 2337, 245, 3111, 4654, 3543, 1282, 4058],
 [1435, 216, 4824, 32, 3024, 4190, 664, 3708],
 [769, 2714, 1987, 2502, 1782, 707, 2256, 1911, 3268, 3121, 4806],
 [3847, 3231, 389, 3111, 1282, 3585],
 [4786, 694, 2805, 4379, 304, 2286, 3873, 251, 1351],
 [216, 1153, 4747, 4281, 4317, 218, 4170, 389, 433, 1627],
 [3941, 3824, 235, 904, 3082, 389, 433, 1627],
 [3872, 2407, 4001, 2898, 2688, 3149, 1384, 3615, 3996, 1835],
 [2921, 4591, 2058],
 [1651, 843, 1292, 496, 1282, 948, 3493, 2815],
 [4036, 2739, 521,

In [27]:
sent_len=20
embed_doc=pad_sequences(onehot_repr,padding='pre',maxlen=sent_len)

In [28]:
print(embed_doc)

[[   0    0    0 ... 4181 1745 1385]
 [   0    0    0 ... 4293 4275 2815]
 [   0    0    0 ...  517 3512 4522]
 ...
 [   0    0    0 ...  389  433 1627]
 [   0    0    0 ... 1405 4210 2491]
 [   0    0    0 ... 2579 2573 4657]]


In [29]:
embed_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  829,
       4353, 4483,  649,  389,  423, 4437, 4181, 1745, 1385])

In [30]:
embeding_vector_feat=40
model=Sequential()
model.add(Embedding(voc_size,embeding_vector_feat,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [32]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
import numpy
x_final=np.array(embed_doc)
y_final=np.array(y)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
x_final.shape,y_final.shape

((18285, 20), (18285,))

In [36]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [38]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 36ms/step - loss: 0.4721 - accuracy: 0.7622 - val_loss: 0.1900 - val_accuracy: 0.9181
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1355 - accuracy: 0.9468 - val_loss: 0.2026 - val_accuracy: 0.9137
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0855 - accuracy: 0.9704 - val_loss: 0.2134 - val_accuracy: 0.9190
Epoch 4/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0614 - accuracy: 0.9801 - val_loss: 0.2620 - val_accuracy: 0.9188
Epoch 5/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0433 - accuracy: 0.9856 - val_loss: 0.2998 - val_accuracy: 0.9195
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0228 - accuracy: 0.9931 - val_loss: 0.3722 - val_accuracy: 0.9175
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0134 - accuracy: 0.9965 - val_loss: 0.5177 - val_accuracy: 0.913

In [46]:
y_pred=model.predict_classes(x_test)

C:\Users\KUMAR HIREMATH\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [47]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [49]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [50]:
confusion_matrix(y_test,y_pred)

array([[3129,  290],
       [ 230, 2386]], dtype=int64)

In [51]:
accuracy_score(y_test,y_pred)

0.9138359569179785